<a href="https://colab.research.google.com/github/Affanamin/RNN_Predicting-Next-Word/blob/master/RNN_Predicting_Next_Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.preprocessing import sequence
import os
import numpy as np


Using TensorFlow backend.


In [3]:
# Read, then decode for py2 compat.
text = open('sample_data/Shakespear.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1155392 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to


In [0]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [6]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [0]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [10]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citize

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen


EXAMPLE

INPUT
:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:

OUTPUT

You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:


In [0]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16896     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 66)            67650     
Total params: 5,331,522
Trainable params: 5,331,522
Non-trainable params: 0
_________________________________________________________________


In [13]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [15]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-5.24993287e-03  3.51384305e-03 -4.39455546e-03 ... -2.00085575e-03
   4.48683091e-03  2.19968753e-03]
 [-2.74243066e-03  7.81201012e-03 -8.14377703e-03 ... -9.72601399e-03
   7.45243579e-03 -2.20288057e-03]
 [-2.49691377e-03  7.57870823e-03 -3.54812690e-03 ... -4.74525942e-03
   3.76986526e-03 -4.78145247e-03]
 ...
 [-8.05741176e-04  1.33438443e-03 -1.16913114e-04 ... -1.19505590e-03
  -8.36926047e-05  4.61265957e-03]
 [-1.92704727e-03  1.50808727e-03  1.64525479e-03 ...  2.37505138e-03
  -3.15607036e-03  2.60789995e-03]
 [ 3.74519499e-03  2.83054053e-03 -3.55140213e-03 ...  7.76270218e-03
  -1.89446320e-03 -1.51895673e-03]], shape=(100, 66), dtype=float32)


In [16]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"J;DjXMPCs,ObjwJ\nReXkro,Tqk:Lcg\nugvF:CT;OHzMm?\ri\nUbjIZbLIXyPPty33- sRCf$reaZNLqNzCJKLDeIw-;XIiF'tGYnK"

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [20]:
history = model.fit(data, epochs=60, callbacks=[checkpoint_callback])

Epoch 1/60
178/178 [==============================] - 10s 55ms/step - loss: 2.4981
Epoch 2/60
178/178 [==============================] - 10s 55ms/step - loss: 1.7924
Epoch 3/60
178/178 [==============================] - 10s 55ms/step - loss: 1.5604
Epoch 4/60
178/178 [==============================] - 9s 53ms/step - loss: 1.4382
Epoch 5/60
178/178 [==============================] - 10s 55ms/step - loss: 1.3640
Epoch 6/60
178/178 [==============================] - 10s 55ms/step - loss: 1.3120
Epoch 7/60
178/178 [==============================] - 10s 55ms/step - loss: 1.2700
Epoch 8/60
178/178 [==============================] - 10s 55ms/step - loss: 1.2330
Epoch 9/60
178/178 [==============================] - 10s 55ms/step - loss: 1.1988
Epoch 10/60
178/178 [==============================] - 10s 54ms/step - loss: 1.1648
Epoch 11/60
178/178 [==============================] - 10s 54ms/step - loss: 1.1315
Epoch 12/60
178/178 [==============================] - 10s 54ms/step - loss: 1.0958
Ep

In [0]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [0]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [25]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))




Type a starting string: ROMEO: 
ROMEO: windows not for charit.
O, as this day shall think no less.
Courage, thou art all: I do not infected marriage
Thou hadst a soul d hence betwixt their loves must fall.

First Lord:
Beseech your hind of mine?
Consider of the memory is gone with us.

BRUTUS:
You saw no it is: which, as he proved bestow
Has ten to one is wink. The garments thou hast be,
All thin and daughter?

PROSPERO:
Shall we perceive the trial instantly.
I saw her looks arm forth one so rares him
now did hear of their poverty. If you
governs for me to speak of Petruchio.'

QUEEN ELIZABETH:
Cousins, indeed; if ever he not know,
Who should be publicly sellen of the fleet
Which I did then been so resolved to help you frame
For this old house of Marcius, that was
the violation of your youth.

SICINIU
